In [1]:
import pathlib as pl
import pandas as pd

summary_phased_folder = pl.Path("/home/ebertp/work/projects/hgsvc/2023_assm_stats/verkko/sseq_phased")
summary_unphased_folder = pl.Path("/home/ebertp/work/projects/hgsvc/2023_assm_stats/verkko/unphased")

summary_out_folder = pl.Path(
    "/home/ebertp/work/projects/hgsvc/2023_assm_stats/verkko"
)

table_out_all_sseq = summary_out_folder.joinpath("assembly_stats_all.verkko.sseq.tsv")
table_out_sub_sseq = summary_out_folder.joinpath("assembly_stats_sub.verkko.sseq.tsv")
table_out_all_unps = summary_out_folder.joinpath("assembly_stats_all.verkko.none.tsv")
table_out_sub_unps = summary_out_folder.joinpath("assembly_stats_sub.verkko.none.tsv")

merged = []
for tsv_file in summary_phased_folder.glob("*.tsv"):
    sample = tsv_file.name.split(".")[0]
    assm_unit = tsv_file.name.split(".")[-3].split("-", 1)[-1]
    df = pd.read_csv(tsv_file, sep="\t", header=0)
    df["sample"] = sample
    df["assembly_unit"] = assm_unit
    df["assembler"] = "verkko"
    merged.append(df)
    
merged = pd.concat(merged, axis=0, ignore_index=False)
merged.drop("source", axis=1, inplace=True)

subset_stats = [
    "total_length_grt_0bp",
    "total_num_grt_0bp",
    "length_auN_grt_0bp",
    "total_length_grt_500kbp",
    "total_num_grt_500kbp",
    "length_auN_grt_500kbp",    
]

subset = merged.loc[merged["statistic"].isin(subset_stats), :].copy()

reorder_columns = ["sample", "assembly_unit", "statistic", "value"]

merged = merged[reorder_columns]
subset = subset[reorder_columns]

merged.sort_values(["sample", "assembly_unit", "statistic"], inplace=True)
with open(table_out_all_sseq, "w") as dump:
    _ = dump.write("# AUTOGEN TABLE - DO NOT REFORMAT\n")
    _ = dump.write("# Verkko Strand-seq phased assemblies\n")
    merged.to_csv(dump, sep="\t", header=True, index=False)

subset.sort_values(["sample", "assembly_unit", "statistic"], inplace=True)
with open(table_out_sub_sseq, "w") as dump:
    _ = dump.write("# AUTOGEN TABLE - DO NOT REFORMAT\n")
    _ = dump.write("# Verkko Strand-seq phased assemblies\n")
    subset.to_csv(dump, sep="\t", header=True, index=False)


